In [ ]:
!pip install --upgrade google-cloud-pubsub

In [ ]:
from google.cloud import pubsub_v1

# Coloque aqui o id do projeto e do topico
project_id = "<id do projeto>"
topic_id = "<id do tópico>"

publisher = pubsub_v1.PublisherClient()
# O método `topic_path` cria um identificador único no formato
# `projects/{project_id}/topics/{topic_id}`
topic_path = publisher.topic_path(project_id, topic_id)

for n in range(1, 10):
    data_str = f"Mensagem {n}"
    # Dados deve estar no formato bytestring
    data = data_str.encode("utf-8")
    # Quando publicar a mensagem, o cliente retorna um objeto future.
    future = publisher.publish(topic_path, data)
    print(future.result())

print(f"Mensagens publicadas no tópico {topic_path}.")

In [ ]:
from concurrent.futures import TimeoutError
from google.cloud import pubsub_v1

# Coloque aqui o id do projeto e da subscrição
project_id = "<id do projeto>"
subscription_id = "<id da subscrição>"

# Número de segundo para o subscriber escutar por mensagens
timeout = 5.0

subscriber = pubsub_v1.SubscriberClient()
# O método `subscription_path` cria um identificador único no formato
# `projects/{project_id}/subscriptions/{subscription_id}`
subscription_path = subscriber.subscription_path(project_id, subscription_id)

def callback(message: pubsub_v1.subscriber.message.Message) -> None:
    print(f"Recebeu a mensagem {message}.")
    message.ack()

streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)
print(f"Escutando por mensagens na subscrição {subscription_path}..\n")

# O block with permite que chame o método close() do subscriber automaticamente no final
with subscriber:
    try:
        # Quando o timeout não é configurado, result() irá bloquear indefinidamente,
        # menos nos casos em que se lança alguma exceção.
        streaming_pull_future.result(timeout=timeout)
    except TimeoutError:
        streaming_pull_future.cancel()  # Dispara o shutdown
        streaming_pull_future.result()  # Bloqueia até que o shutdown aconteça